In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'mesolitica-tpu.json'

In [2]:
from glob import glob
import tensorflow as tf
from tqdm import tqdm
import malaya_speech
from malaya_speech.utils import subword
import numpy as np
import mp
from google.cloud import storage
from unidecode import unidecode

/home/ubuntu/.local/lib/python3.7/site-packages/malaya_boilerplate/frozen_graph.py:25: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0
  'Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0'


In [4]:
subwords = subword.load('transducer-singlish.subword')

In [5]:
singlish = glob(f'part3-splitted/wav/*.wav')
len(singlish)

1261328

In [7]:
import unicodedata
import re
import itertools

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]

def preprocessing_text(string):
    
    string = unicodedata.normalize('NFC', string.lower())
    string = string.replace('\'', '')
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [8]:
def get_after_mandarin(word):
    if '<mandarin>' in word:
        w = word.split('>')[1].split(':')[1]
        return w.split('</')[0]
    else:
        return word
    
def get_before_mandarin(word):
    if '</mandarin>' in word:
        return word.split('</')[0]
    else:
        return word

def replace_paralinguistic(string, replaces = ['(ppb)', '(ppc)', '(ppl)', '(ppo)', '<UNK>', '<MANDARIN>']):
    for r in replaces:
        string = string.replace(r, ' ')
    string = string.split()
    string = [get_after_mandarin(w) for w in string]
    string = [get_before_mandarin(w) for w in string]
    string = [w for w in string if w[0] not in '<[(' and w[-1] not in '>])']
    return ' '.join(string)

In [12]:
def loop(files):
    files, index = files
    results = []
    for i in tqdm(files):
        try:
            p = i.replace('/wav','/text')
            with open(f'{p}.txt') as fopen:
                text = fopen.read()
            if len(text) < 2:
                continue
            if text[0] == '<' and text[-1] == '>':
                continue
            text = replace_paralinguistic(text)
            text = preprocessing_text(text)
            if len(text):
                results.append((i, text))
        except Exception as e:
            pass
    return results

In [13]:
loop((singlish[:10], 0))

100%|██████████| 10/10 [00:00<00:00, 9387.43it/s]


[('part3-splitted/wav/3071-2-0-1656.wav',
  'fear you know when you watch your family like divorce and all then you realise that the main problem of their'),
 ('part3-splitted/wav/3109-1-0-2607.wav', 'bring to hall'),
 ('part3-splitted/wav/3232-1-0-1623.wav', 'very very bad')]

In [15]:
singlishs = mp.multiprocessing(singlish, loop, cores = 12)

100%|██████████| 105110/105110 [02:27<00:00, 711.07it/s] 


In [16]:
len(singlishs)

691624

In [17]:
import six

def to_example(dictionary):
    """Helper: build tf.Example from (string -> int/float/str list) dictionary."""
    features = {}
    for (k, v) in six.iteritems(dictionary):
        if not v:
            raise ValueError('Empty generated field: %s' % str((k, v)))
        # Subtly in PY2 vs PY3, map is not scriptable in py3. As a result,
        # map objects will fail with TypeError, unless converted to a list.
        if six.PY3 and isinstance(v, map):
            v = list(v)
        if isinstance(v[0], six.integer_types) or np.issubdtype(
            type(v[0]), np.integer
        ):
            features[k] = tf.train.Feature(
                int64_list=tf.train.Int64List(value=v)
            )
        elif isinstance(v[0], float):
            features[k] = tf.train.Feature(
                float_list=tf.train.FloatList(value=v)
            )
        elif isinstance(v[0], six.string_types):
            if not six.PY2:  # Convert in python 3.
                v = [bytes(x, 'utf-8') for x in v]
            features[k] = tf.train.Feature(
                bytes_list=tf.train.BytesList(value=v)
            )
        elif isinstance(v[0], bytes):
            features[k] = tf.train.Feature(
                bytes_list=tf.train.BytesList(value=v)
            )
        else:
            raise ValueError(
                'Value for %s is not a recognized type; v: %s type: %s'
                % (k, str(v[0]), str(type(v[0])))
            )
    return tf.train.Example(features=tf.train.Features(feature=features))

In [18]:
sr = 16000
maxlen = 18
minlen_text = 1
global_count = 0

In [19]:
def loop(files):
    client = storage.Client()
    bucket = client.bucket('mesolitica-tpu-general')
    files, index = files
    output_file = f'{index}-{global_count}.tfrecord'
    writer = tf.io.TFRecordWriter(output_file)
    for s in tqdm(files):
        try:
            if len(s[1]) < minlen_text:
                continue
            y, _ = malaya_speech.load(s[0])
            if (len(y) / sr) > maxlen:
                continue
            t = subword.encode(subwords, s[1], add_blank=False)
            example = to_example({'waveforms': y.tolist(), 
                                  'targets': t, 
                                  'targets_length': [len(t)]})
            writer.write(example.SerializeToString())
        except Exception as e:
            print(e)
            pass
    writer.close()
    blob = bucket.blob(f'imda/part3/{output_file}')
    blob.upload_from_filename(output_file)
    os.system(f'rm {output_file}')

In [20]:
loop((singlishs[:10], 0))

100%|██████████| 10/10 [00:00<00:00, 31.52it/s]


In [21]:
batch_size = 25000
for i in range(0, len(singlishs), batch_size):
    batch = singlishs[i: i + batch_size]
    mp.multiprocessing(batch, loop, cores = 6, returned = False)
    global_count += 1

  3%|▎         | 123/4166 [00:03<01:36, 42.06it/s]

zero-size array to reduction operation maximum which has no identity


  6%|▌         | 232/4166 [00:06<02:02, 32.17it/s]

zero-size array to reduction operation maximum which has no identity


 13%|█▎        | 522/4166 [00:14<01:40, 36.23it/s]

...

zero-size array to reduction operation maximum which has no identity

 89%|████████▉ | 2468/2770 [01:09<00:10, 28.43it/s]

 98%|█████████▊| 2718/2770 [01:15<00:01, 32.98it/s]

zero-size array to reduction operation maximum which has no identity


100%|██████████| 4/4 [00:00<00:00, 66.78it/s]


In [22]:
from malaya_speech.utils import tf_featurization

config = malaya_speech.config.transducer_featurizer_config
featurizer = tf_featurization.STTFeaturizer(**config)

In [23]:
n_mels = 80

def preprocess_inputs(example):
    s = featurizer.vectorize(example['waveforms'])
    mel_fbanks = tf.reshape(s, (-1, n_mels))
    example['inputs'] = mel_fbanks
    return example

def parse(serialized_example):

    data_fields = {
        'waveforms': tf.compat.v1.VarLenFeature(tf.float32),
        'targets': tf.compat.v1.VarLenFeature(tf.int64),
        'targets_length': tf.compat.v1.VarLenFeature(tf.int64),
    }
    features = tf.compat.v1.parse_single_example(
        serialized_example, features = data_fields
    )
    for k in features.keys():
        features[k] = features[k].values
        
    features = preprocess_inputs(features)

    keys = list(features.keys())
    for k in keys:
        if k not in ['waveforms', 'inputs', 'targets', 'targets_length']:
            features.pop(k, None)

    return features

def get_dataset(files, batch_size = 2, shuffle_size = 32, thread_count = 24):
    def get():
        dataset = tf.data.TFRecordDataset(files)
        dataset = dataset.shuffle(shuffle_size)
        dataset = dataset.map(parse, num_parallel_calls = thread_count)
        dataset = dataset.repeat()
        return dataset

    return get

In [24]:
files = tf.io.gfile.glob('gs://mesolitica-tpu-general/imda/part3/*.tfrecord')
d = get_dataset(files)()
d = d.as_numpy_iterator()

In [25]:
next(d)

{'targets': array([ 23,   2,  12, 795,  77,  56,   1, 795,  75,  23,   2,  12, 795,
         77,  56, 150, 795, 100, 210, 269, 329, 131, 795,  23,   2, 225,
        610]),
 'targets_length': array([27]),
 'waveforms': array([-0.00272823, -0.00335782, -0.00062959, ...,  0.00818468,
         0.01112277,  0.00818468], dtype=float32),
 'inputs': array([[-2.8447442 , -2.780432  , -2.6907475 , ..., -0.80764127,
         -0.9331325 , -1.3005956 ],
        [-3.197297  , -2.97263   , -2.7670536 , ..., -0.74995244,
         -0.7892152 , -1.0798239 ],
        [-2.6084573 , -2.61795   , -2.6498468 , ..., -0.80574274,
         -0.9831676 , -1.1573235 ],
        ...,
        [-2.725497  , -2.4747465 , -2.2584088 , ..., -0.2696587 ,
         -0.43722332, -0.7711975 ],
        [-2.7993689 , -2.8822708 , -3.2491498 , ..., -0.21634445,
         -0.4464349 , -0.69611686],
        [-2.6580067 , -2.6457634 , -2.6325219 , ..., -0.2595962 ,
         -0.15352216, -0.6988079 ]], dtype=float32)}